In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
#from scipy import stats
from datetime import datetime as dt, timedelta
from IPython.display import display, HTML
import functions as f

%matplotlib inline
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [2]:
filename = './data/Master_Data_Union_2014_2018_B_Recency_Pre2014.txt'

In [3]:
%%time
chunksize = 10**6
chunks = pd.read_csv(filename, chunksize=chunksize, sep='|',low_memory=False)
#for chunk in pd.read_csv(filename,chunksize=chunksize,sep='|',low_memory=False):
    #DF = pd.concat([DF,chunk])

CPU times: user 3.98 ms, sys: 6.26 ms, total: 10.2 ms
Wall time: 28 ms


In [4]:
%%time
df = chunks.get_chunk(1000000)

CPU times: user 15.3 s, sys: 9.94 s, total: 25.3 s
Wall time: 36.9 s


In [5]:
df.head()

,BIRTH_DATE,CHANNEL,CLOSEST_STORE1_DIST,DAY_OF_THE_WK,FIRST_CATALOG_ORDER_DATE,FIRST_ORDER_CHANNEL,FIRST_ORDER_DATE,FIRST_RETAIL_ORDER_DATE,FIRST_WEB_ORDER_DATE,FISCAL_DAY_OF_THE_YR,FISCAL_MO,FISCAL_QTR,FISCAL_WK,FISCAL_YR,ILINK,IS_CALLABLE_IND,IS_EMAILABLE_IND,IS_JJCH_IND,JJCH_CLOSE_DATE,JJCH_OPEN_DATE,MASTER_CHANNEL,ORDER_DATE,ORDER_DETAIL_KEY,ORDER_KEY,ORIGINAL_RETAIL_PRICE_AMT,PRICE_CD,ORDER_SHIPPING_AMT,PAY_TYPE_CD,SHIPPED_COST_AMT,SHIPPED_GROSS_AMT,SHIPPED_QTY,SHIPPED_SOLD_AMT,SKU_CD,STATE,STYLE_NB,ZIP,FABRIC_CATEGORY_DESC,BODY_NB,DESCRIPTION,EXTENDED_SIZE_DESC,COLOR_FAMILY_DESC,END_USE_DESC,STYLETYPEDESC,CLASS_NAME,DEPARTMENT_NAME,Source,Days_Lapsed,Days_Lapsed_Known,Days_Lapsed_LB,Recency_Month,Recency_Quarter,Recency_Year,PRIOR_ORDER_DATE,margin,discount,markdown,discount_plus
0,1967-06-29,W,20.5,Monday,2005-06-10,R,2005-05-18,2005-05-18,2017-01-02,338,12,4,49,2016,4,Y,Y,NaN,NaN,NaN,D,2017-01-02,6258484654,7094417803,79.0,SP,0.0,MC,16.1729,49.99,1,35.99,07569114XL,MA,75691,1473,Cotton/Cotton Bl,75691,Plaid Double-Cloth Long Tunic,Misses,Blue,Core,Tops,Long Sleeve Shirts,Woven Shirts,3,4247,0,1066.0,React,React,React,6/10/2005,19.8171,14.0,29.01,43.01
1,1967-06-29,W,20.5,Monday,2005-06-10,R,2005-05-18,2005-05-18,2017-01-02,338,12,4,49,2016,4,Y,Y,NaN,NaN,NaN,D,2017-01-02,6256936659,7094417803,79.0,SP,0.0,MC,16.1900,59.99,1,35.99,07569114XXL,MA,75691,1473,Cotton/Cotton Bl,75691,Plaid Double-Cloth Long Tunic,Misses,Blue,Core,Tops,Long Sleeve Shirts,Woven Shirts,3,4247,0,1066.0,React,React,React,6/10/2005,19.8000,24.0,19.01,43.01
2,1967-06-29,W,20.5,Monday,2005-06-10,R,2005-05-18,2005-05-18,2017-01-02,338,12,4,49,2016,4,Y,Y,NaN,NaN,NaN,D,2017-01-02,6256936661,7094417803,79.0,SP,0.0,MC,18.6200,39.99,1,23.99,0808010OE1SZ,MA,80801,1473,Wool/Wool Bl,80801,Terrace Scroll Scarf,No Ext Size,Red,Core,Accessories,Other Scarves,Fashion Scarves,3,4247,0,1066.0,React,React,React,6/10/2005,5.3700,16.0,39.01,55.01
3,1967-06-29,W,20.5,Monday,2005-06-10,R,2005-05-18,2005-05-18,2017-01-02,338,12,4,49,2016,4,Y,Y,NaN,NaN,NaN,D,2017-01-02,6258484652,7094417803,89.0,SP,0.0,MC,NaN,0.00,0,0.00,075781LLBL,MA,75781,1473,Synthetic/Syn Blend,75781,Grace Sweater Tunic,Misses,Red,Core,Tops,Pullover Sweaters,Sweaters,3,4247,0,1066.0,React,React,React,6/10/2005,NaN,0.0,89.00,89.00
4,1967-06-29,W,20.5,Monday,2005-06-10,R,2005-05-18,2005-05-18,2017-01-02,338,12,4,49,2016,4,Y,Y,NaN,NaN,NaN,D,2017-01-02,6256936660,7094417803,89.0,SP,0.0,MC,20.1079,69.99,1,41.99,075781LLBXL,MA,75781,1473,Synthetic/Syn Blend,75781,Grace Sweater Tunic,Misses,Red,Core,Tops,Pullover Sweaters,Sweaters,3,4247,0,1066.0,React,React,React,6/10/2005,21.8821,28.0,19.01,47.01


In [ ]:
#only selecting orders that happened between may 2017-july 2017 for sample size

In [6]:
%%time
DF_SAMPLE = df.loc[df.IS_EMAILABLE_IND=='Y']
DF = DF_SAMPLE.copy()
DF['ORDER_DATE'] = DF.ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d'))
df1 = DF.loc[(DF.ORDER_DATE >= dt(2017,5,1).date()) & 
              (DF.ORDER_DATE < dt(2017,8,1).date())]
df2 = DF.loc[(DF.ORDER_DATE >= dt(2017,5,1).date()) & 
              (DF.ORDER_DATE < dt(2017,6,1).date())]
df3 = DF.loc[(DF.ORDER_DATE >= dt(2017,6,1).date()) & 
              (DF.ORDER_DATE < dt(2017,7,1).date())]
df4 = DF.loc[(DF.ORDER_DATE >= dt(2017,7,1).date()) & 
              (DF.ORDER_DATE < dt(2017,8,1).date())]
#df1 = df1.loc[(df1.ORDER_DATE.dt.month >= 5) & (df1.ORDER_DATE.dt.month < 8)]
#df1 = df1[~df1.BIRTH_DATE.isnull()]

CPU times: user 14.2 s, sys: 2.57 s, total: 16.8 s
Wall time: 20.1 s


In [285]:
#print 'unique customer ids:', len(df1.ILINK.unique())
#print 'Number emailable:', df1.shape[0]

unique customer ids: 5597
Number emailable: 40180


In [286]:
df1['BIRTH_DATE'] = df1.BIRTH_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['FIRST_CATALOG_DATE'] = df1.FIRST_CATALOG_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['ORDER_DATE'] = df1.ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
df1['FIRST_ORDER_DATE'] = df1.FIRST_ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['FIRST_RETAIL_ORDER_DATE'] = df1.FIRST_RETAIL_ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['FIRST_WEB_ORDER_DATE'] = df1.FIRST_WEB_ORDER_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['JJCH_CLOSE_DATE'] = df1.JJCH_CLOSE_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['JJCH_OPEN_DATE'] = df1.JJCH_OPEN_DATE.apply(lambda x: dt.strptime(x,'%Y-%m-%d').date())
#df1['PRIOR_ORDER_DATE'] = df1.PRIOR_ORDER_DATE.apply(lambda x: dt.strptime(x,'%m/%d/%Y').date())

# Definitions

In [7]:
def normalize(l):
    s = sum(l)
    if s == 0:
        return [0]*8
    else:
        return [i/s for i in l]

##standardize variables
def standardization_f(x):
    x_bar = np.mean(x)
    s = np.std(x)
    if s == 0:
        return 0
    else:
        x_z = (x-x_bar)/s
    return(x_z)

In [9]:
#make empty dataframe to add onto with customer features
"""columns = ['Age',
           'WovenShirts_N','KnitTops_N','Pants_N','Dresses_N',
           'WovenShirts_AvgN_Order','KnitTops_AvgN_Order','Pants_AvgN_Order','Dresses_AvgN_Order',
           'WovenShirts_Total','KnitTops_Total','Pants_Total','Dresses_Total',
           'WovenShirts_Amt_Order','KnitTops_Amt_Order','Pants_Amt_Order','Dresses_Amt_Order',
           'WovenShirts_AvgAmt','KnitTops_AvgAmt','Pants_AvgAmt','Dresses_AvgAmt']"""
''

''

# Exploration


In [10]:
#reload(f)

In [ ]:
#Get Pref/rating over course of May 1 2017 - July 31 2017

In [11]:
#ilinks = df1.ILINK.unique().tolist()
#users = ilinks[:1000] #sample 

In [12]:
#For sample size, I'm looking at customers currently that have at least purcahse in May-June and June-July Months
mergedf = df2.merge(df3,on='ILINK')
ilinks = mergedf.ILINK.unique().tolist()
users = ilinks[:1000]#sample
print len(ilinks)

1286


In [13]:
%%time
#The features include
cols = ['Ilink','N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
        'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
        'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
        'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
#cols2 = ['Ilink','N_ws','S_ws','s_ws','si_ws','R_ws',
#        'N_kt','S_kt','s_kt','si_kt','R_kt',
#        'N_d','S_d','s_d','si_d','R_d',
#        'N_p','S_p','s_p','si_p','R_p']
#cols = ['Ilink','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
#       'S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
#        'S_d','s_d','si_d','D_d','d_d','di_d','R_d',
#        'S_p','s_p','si_p','D_p','d_p','di_p','R_p']
table = pd.DataFrame([],columns=cols)

#need to create rows that describe a customers freq and sales of items in specific departments
#This also builds a customer "sigma" value that will store its information on feature proportions
depts = ['Woven Shirts','Knit Tops','Dresses','Pants']
K = 4 #Number of Depts/topics
UserSigma = {}
for ilink in users:
    dfCust = df1.loc[(df1.ILINK==ilink)]
    N_u = dfCust.shape[0]#sum([f.getTotalFreq(dfCust,dept) for dept in depts])
    fts = []
    #fts2 = []
    UserSigma[ilink] = {}
    for dept in depts:
        N = f.getTotalFreq(dfCust,dept)
        n = f.getAvgFreqOrder(dfCust,dept)
        O,Q,G,S,D = f.getTotalAmts(dfCust,dept)
        o,q,g,s,d = f.getAvgAmtsOrder(dfCust,dept)
        oi,qi,gi,si,di = f.getAvgAmtItem(dfCust,dept)
        alpha = [N,n,S,s,si,D,d,di]
        #alpha = [S,s,si,D,d,di]
        #alpha = [N,S,s,si]
        alpha = [0 if math.isnan(x) else x for x in alpha]
        alpha1 = np.array(normalize(alpha))
        sigma = (N + alpha1)/(N_u + K*alpha1)
        UserSigma[ilink][dept] = sigma
        pref = np.dot(sigma,alpha1).round(2)
        #print ilink, alpha1
        #print sigma
        #print ''
        fts.append(alpha+[pref])
        #fts2.append(alpha)
    
    """u,s,vt = np.linalg.svd(fts2)
    for i,row in enumerate(fts2):
        fts2[i] = fts2[i] + [np.dot(u[i],vt[i]).round(3)]"""
    row = [i for sub in fts for i in sub]
    row.insert(0,ilink)
    temp = pd.DataFrame([tuple(row)],columns=cols)
    table = pd.concat([table,temp])

CPU times: user 2min 28s, sys: 2.6 s, total: 2min 30s
Wall time: 2min 38s


In [14]:
cols = ['N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
        'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
        'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
        'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
table = table.loc[~(table[cols]==0).all(axis=1)]

In [15]:
table.reset_index(drop=True,inplace=True)

In [16]:
table.head()

,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
0,800,3,2.0,139.12,69.56,46.37,77.88,38.94,25.96,0.67,0,0.0,0.00,0.00,0,0.00,0.00,0,0,0,0.0,0.0,0.00,0,0.0,0.0,0,0,0,0.0,0.00,0.00,0,0.00,0.00,0,0
1,825,1,0.0,34.19,8.55,34.19,25.80,6.45,25.80,0.08,2,0.0,46.20,11.55,23.1,32.79,8.20,16.39,0.15,0,0.0,0.0,0.00,0,0.0,0.0,0,0,4,1.0,138.87,34.72,34.72,100.10,25.02,25.02,0.28
2,845,8,3.0,292.99,97.66,36.62,162.00,54.00,20.25,0.22,4,1.0,118.75,39.58,29.69,69.24,23.08,17.31,0.11,0,0.0,0.0,0.00,0,0.0,0.0,0,0,14,5.0,514.21,171.40,36.73,265.79,88.60,18.99,0.37
3,872,1,0.0,52.53,6.57,52.53,26.47,3.31,26.47,0.05,13,2.0,437.99,54.75,33.69,174.95,21.87,13.46,0.52,1,0.0,119.0,14.88,119,0.0,0.0,0,0.06,4,0.0,191.28,23.91,47.82,95.71,11.96,23.93,0.17
4,896,8,1.0,343.59,28.63,42.95,140.41,11.70,17.55,0.15,34,3.0,940.48,78.37,27.66,359.46,29.96,10.57,0.6,0,0.0,0.0,0.00,0,0.0,0.0,0,0,6,0.0,293.65,24.47,48.94,111.35,9.28,18.56,0.11


# Investigating If R has any substance/quality to Month by Month 

## Functions to be used for checking

In [98]:
#function to get table from a list of ilinks and a dataframe/dataset from the month
def getTableRating(ilinks,dataframe):
    cols = ['Ilink','N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
        'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
        'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
        'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
    tab = pd.DataFrame([],columns=cols)
    #need to create rows that describe a customers freq and sales of items in specific departments
    depts = ['Woven Shirts','Knit Tops','Dresses','Pants']
    K = 4 #Number of Depts/topics
    for ilink in ilinks:
        dfCust = dataframe.loc[(dataframe.ILINK==ilink)]
        if ~dfCust.empty:
            N_u = dfCust.shape[0]#sum([f.getTotalFreq(dfCust,dept) for dept in depts])
            #print ilink,N_u
            fts = []
            for dept in depts:
                N = f.getTotalFreq(dfCust,dept)
                n = f.getAvgFreqOrder(dfCust,dept)
                O,Q,G,S,D = f.getTotalAmts(dfCust,dept)
                o,q,g,s,d = f.getAvgAmtsOrder(dfCust,dept)
                oi,qi,gi,si,di = f.getAvgAmtItem(dfCust,dept)
                alpha = [N,n,S,s,si,D,d,di]
                #alpha = [S,s,si,D,d,di]
                #alpha = [N,S,s,si]
                alpha = [0 if math.isnan(x) else x for x in alpha]
                alpha1 = np.array(normalize(alpha))
                sigma = UserSigma[ilink][dept]
                #sigma = (N + alpha1)/(N_u + K*alpha1)
                pref = np.dot(sigma,alpha1).round(2)
                #print ilink,alpha
                #print sigma
                #print ''
                fts.append(alpha+[pref])
                #fts2.append(alpha)
        else:
            print ilink
        """u,s,vt = np.linalg.svd(fts2)
        for i,row in enumerate(fts2):
            fts2[i] = fts2[i] + [np.dot(u[i],vt[i]).round(3)]"""
        row = [i for sub in fts for i in sub]
        row.insert(0,ilink)
        temp = pd.DataFrame([tuple(row)],columns=cols)
        tab = pd.concat([tab,temp])
    return tab

In [20]:
#This function will get the customers highest Rating pref
#It checks the rating first
#if ratings are equal it checks for freq, total sold/sales as tiebreakers in that order
#if that is not a tie-breaker it will return as many dept as "preferred" and consider that user as 
#having multiple preferences
def getHigherPref(row):
    ws = (row.R_ws,row.N_ws,row.S_ws)
    kt = (row.R_kt,row.N_kt,row.S_kt)
    d = (row.R_d,row.N_d,row.S_d)
    p = (row.R_p,row.N_p,row.S_p)
    R = {'Woven Shirts': ws, 'Knit Tops': kt, 'Dresses': d, 'Pants': p}
    maxNum = 0.0
    maxKey = []
    #get those with highest rating first
    for key,value in R.iteritems():
        if value[0] > maxNum:
            maxNum = value[0]
            maxKey.append(key)
        elif value[0] == maxNum and value[0] != 0:
            maxKey.append(key)
    
    #if no purchases were made
    if len(maxKey) == 0:
        return 0
    #if rating are equal, then look at freq purchased for pref
    i = 1
    while len(maxKey) != 1:
        RTop = {x:R[x] for x in maxKey}
        n = 0
        maxKey = []
        for key,value in RTop.iteritems():
            #print value[i]
            if value[i] > n:
                n = value[i]
                if len(maxKey)==0:
                    maxKey.append(key)
                else:
                    maxKey.pop()
                    maxKey.append(key)
            elif value[i] == n and value[i] !=0:
                maxKey.append(key)
        #print maxKey,n
        if len(maxKey) > 1:
            i += 1
            if i > 2:
                return maxKey
    return maxKey[0]

## 1. First we get what Users pref for May 1, 2017 - August 1,2017

In [21]:
%%time
#Store User Ilinks based on their dept. preference
Users = {'Woven Shirts': [],
         'Knit Tops': [],
         'Dresses': [],
         'Pants': []}
N=0 #just a number to help me track progress
for i,row in table.iterrows():
    if N%100 == 0:
        print N
    N+=1
    
    #print row.Ilink
    key = getHigherPref(row)
    
    #if "key" or depts is multiple it'll store User pref multiple times
    #else if it returns empty key=0, it'll skip the cust
    #else it will store user's pref into dept
    if type(key) is list:
        for x in key:
            Users[x].append(row.Ilink)
    elif key == 0:
        pass
    else:
        Users[key].append(row.Ilink)

0
100
200
300
400
500
600
700
800
900
CPU times: user 428 ms, sys: 58.7 ms, total: 487 ms
Wall time: 462 ms


In [59]:
#table.Ilink.tolist()

#We will check for Woven shirts, Knit Tops, Pants, and Dresses
depts = ['Woven Shirts','Knit Tops','Dresses','Pants']

#this gets sample list of users based on their "proxy pref/rating" based on their Purchase History in May - July 
ilink_ex = {x:Users[x] for x in depts}
#len(ilink_ex[0])

In [85]:
#Give you the relative number of customers that prefer based on proxy rating those categories from May-July timeframe
#Example: based on purchase history from May 2017 - July 2017, the proxy rating says 108 customers prefer Woven Shirts
for dept in depts:
    print "Table Size for %s: %d" % (dept,len(Users[dept]))

Table Size for Woven Shirts: 115
Table Size for Knit Tops: 489
Table Size for Dresses: 121
Table Size for Pants: 272


## 2. Look at these Users in May-June

In [45]:
%%time
table2 = {}
for dept in depts:
    table2[dept] = getTableRating(ilink_ex[dept],df2)
    cols = ['N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
            'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
            'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
            'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
    table2[dept] = table2[dept].loc[~(table2[dept][cols]==0).all(axis=1)] #gets rid of customers that did not purchase anything,but have orders
    table2[dept].reset_index(drop=True,inplace=True)

CPU times: user 1min 8s, sys: 408 ms, total: 1min 8s
Wall time: 1min 9s


In [49]:
#displays first 5 rows of sorted top pref to get a sense of populations features to ratings
#Each table consists of customers that were listed as "preferred" in the categories
#Based on their "proxy rating" for May-July
#For just May month, we apply same proxy rating
pref = {'Woven Shirts': 'R_ws','Knit Tops':'R_kt','Dresses':'R_d','Pants':'R_p'}
for dept in depts:
    print "Table for %s" % dept
    display(table2[dept].sort_values(pref[dept],ascending=False).head())

Table for Woven Shirts


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
68,24256,2,2.0,97.98,97.98,48.99,42.00,42.00,21.00,0.90,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0
25,9004,4,1.0,152.91,38.23,38.23,77.05,19.26,19.26,0.87,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0
62,21942,2,2.0,133.50,133.50,66.75,44.50,44.50,22.25,0.84,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0
78,28815,1,1.0,49.99,49.99,49.99,0.00,0.00,0.00,0.74,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0
5,5100,11,4.0,560.46,186.82,50.95,194.48,64.83,17.68,0.73,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0


Table for Knit Tops


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
299,25968,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,2.0,274.61,68.65,30.51,138.37,34.59,15.37,0.92,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
448,36070,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,6.0,234.42,234.42,39.07,94.58,94.58,15.76,0.90,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
272,23327,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2.0,102.60,102.60,51.30,5.40,5.40,2.70,0.90,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
122,10723,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,3.0,201.31,100.66,33.55,102.69,51.34,17.11,0.89,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,25.93,12.96,25.93,13.07,6.54,13.07,0.09
47,5565,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2.0,81.22,81.22,40.61,32.78,32.78,16.39,0.89,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00


Table for Dresses


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
105,34202,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2.0,101.98,101.98,50.99,68.00,68.00,34,0.92,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
78,26566,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2.0,131.66,131.66,65.83,66.34,66.34,33.17,0.88,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
72,24968,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,65.83,65.83,65.83,33.17,33.17,33.17,0.86,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
83,27850,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,46.54,46.54,46.54,23.45,23.45,23.45,0.81,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
44,16160,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,37.79,37.79,37.79,22.20,22.20,22.2,0.81,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0


Table for Pants


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
102,14649,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,1,1.0,45.88,45.88,45.88,23.12,23.12,23.12,0.89
118,17494,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,1,1.0,22.79,22.79,22.79,17.20,17.20,17.20,0.87
196,28796,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,1,1.0,39.23,39.23,39.23,19.77,19.77,19.77,0.86
95,13741,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,2,2.0,133.50,133.50,66.75,44.50,44.50,22.25,0.84
93,13577,1,1.0,63.41,63.41,63.41,25.59,25.59,25.59,0.14,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,5,5.0,175.96,175.96,35.19,111.02,111.02,22.20,0.82


In [86]:
#Give you the relative number of customers that prefer based on proxy rating those categories from May-June timeframe
#Example: based on purchase history from May 1 2017 - June 1 2017, the proxy rating says 108 customers prefer Woven Shirts
for dept in depts:
    print "Table Size for %s: %d" % (dept,table2[dept].shape[0])

Table Size for Woven Shirts: 108
Table Size for Knit Tops: 463
Table Size for Dresses: 116
Table Size for Pants: 257


In [77]:
#Now we check and see just how many of these N number of customers actually did "prefer" the dept the 3-month timespan said they would
for dept in depts:
    May_Users = {'Woven Shirts': [],
         'Knit Tops': [],
         'Dresses': [],
         'Pants': []}
    N=0
    for i,row in table2[dept].iterrows():
        #if N%100 == 0:
        #    print N
        #N+=1
        #print row.Ilink
        key = getHigherPref(row)
        if type(key) is list:
            for x in key:
                May_Users[x].append(row.Ilink)
        elif key == 0:
            pass
        else:
            May_Users[key].append(row.Ilink)        
    not_depts = [x for x in depts if x != dept]
    deptN = table2[dept].shape[0]
    dept_total = len(May_Users[dept])
    not_dept_total = 0
    for ndept in not_depts:
        not_dept_total += len(May_Users[ndept])
    print "Based on those that prefer %s" % dept
    print "Total Number that still 'prefer' %s: %d" % (dept,dept_total)
    print "Total Number that did not 'prefer' %s: %d" % (dept,not_dept_total)
    print "Ratio of Not_Prefer/Prefer: %f" % (not_dept_total*1.0/dept_total)
    print "Percent that kept %s as pref: %f" %(dept,dept_total*1.0/deptN)
    print ''

Based on those that prefer Woven Shirts
Total Number that still 'prefer' Woven Shirts: 86
Total Number that did not 'prefer' Woven Shirts: 22
Ratio of Not_Prefer/Prefer: 0.255814
Percent that kept Woven Shirts as pref: 0.796296

Based on those that prefer Knit Tops
Total Number that still 'prefer' Knit Tops: 373
Total Number that did not 'prefer' Knit Tops: 93
Ratio of Not_Prefer/Prefer: 0.249330
Percent that kept Knit Tops as pref: 0.805616

Based on those that prefer Dresses
Total Number that still 'prefer' Dresses: 87
Total Number that did not 'prefer' Dresses: 29
Ratio of Not_Prefer/Prefer: 0.333333
Percent that kept Dresses as pref: 0.750000

Based on those that prefer Pants
Total Number that still 'prefer' Pants: 178
Total Number that did not 'prefer' Pants: 81
Ratio of Not_Prefer/Prefer: 0.455056
Percent that kept Pants as pref: 0.692607



## 3. Look at June-July

In [78]:
%%time
table3 = {}
for dept in depts:
    table3[dept] = getTableRating(ilink_ex[dept],df3)
    cols = ['N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
            'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
            'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
            'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
    table3[dept] = table3[dept].loc[~(table3[dept][cols]==0).all(axis=1)] #gets rid of customers that did not purchase anything,but have orders
    table3[dept].reset_index(drop=True,inplace=True)

CPU times: user 1min 11s, sys: 605 ms, total: 1min 12s
Wall time: 1min 15s


In [79]:
#displays first 5 rows of sorted top pref to get a sense of populations features to ratings
#Each table consists of customers that were listed as "preferred" in the categories
#Based on their "proxy rating" for May-July
#For just June month, we apply same proxy rating
pref = {'Woven Shirts': 'R_ws','Knit Tops':'R_kt','Dresses':'R_d','Pants':'R_p'}
for dept in depts:
    print "Table for %s" % dept
    display(table3[dept].sort_values(pref[dept],ascending=False).head())

Table for Woven Shirts


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
27,9004,1,1.0,23.74,23.74,23.74,26.25,26.25,26.25,0.92,0,0.0,0.0,0.00,0,0.0,0.00,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.00,0.00,0,0.00,0.00,0,0
69,24256,7,4.0,444.35,222.18,63.48,158.65,79.32,22.66,0.89,0,0.0,0.0,0.00,0,0.0,0.00,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,1,0.0,51.75,25.88,51.75,17.25,8.62,17.25,0.07
64,21942,1,1.0,62.30,62.30,62.30,26.70,26.70,26.70,0.85,0,0.0,0.0,0.00,0,0.0,0.00,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.00,0.00,0,0.00,0.00,0,0
79,28815,1,1.0,34.99,34.99,34.99,15.00,15.00,15.00,0.80,0,0.0,0.0,0.00,0,0.0,0.00,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.00,0.00,0,0.00,0.00,0,0
5,5100,4,1.0,199.33,49.83,49.83,78.65,19.66,19.66,0.73,2,0.0,96.6,24.15,48.3,41.4,10.35,20.7,0.1,0,0.0,0.0,0.0,0,0.0,0.0,0,0,0,0.0,0.00,0.00,0,0.00,0.00,0,0


Table for Knit Tops


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
301,25968,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,1,1.0,44.84,44.84,44.84,14.16,14.16,14.16,0.95,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00
274,23327,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,2,2.0,93.10,93.10,46.55,4.90,4.90,2.45,0.90,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00
42,5565,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,3,3.0,147.48,147.48,49.16,59.52,59.52,19.84,0.89,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00
449,36070,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,6,3.0,344.84,172.42,57.47,30.15,15.08,5.02,0.88,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00
116,10723,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,11,2.0,345.20,69.04,31.38,138.00,27.60,12.55,0.88,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,1,0.0,25.93,5.19,25.93,13.07,2.61,13.07,0.09


Table for Dresses


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
101,34202,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,2,2.0,152.60,152.60,76.30,65.40,65.40,32.70,0.91,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
74,26566,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,2,2.0,118.36,118.36,59.18,59.64,59.64,29.82,0.88,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
68,24968,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,1,1.0,69.99,69.99,69.99,0.00,0.00,0.00,0.82,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
40,16160,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,1,1.0,39.89,39.89,39.89,20.10,20.10,20.10,0.81,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
87,29254,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,1,1.0,84.79,84.79,84.79,34.21,34.21,34.21,0.80,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0


Table for Pants


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
99,14649,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,2,1.0,79.13,39.56,39.56,58.87,29.44,29.44,0.88
113,17494,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,2,2.0,47.48,47.48,23.74,52.50,52.50,26.25,0.86
91,13741,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,1,1.0,66.75,66.75,66.75,22.25,22.25,22.25,0.85
89,13577,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,2,2.0,93.10,93.10,46.55,44.90,44.90,22.45,0.82
195,28796,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0,2,2.0,121.60,121.60,60.8,6.40,6.40,3.2,0.80


In [87]:
#Give you the relative number of customers that prefer based on proxy rating those categories from June-July timeframe
#Example: based on purchase history from June 2017 - July 2017, the proxy rating says 107 customers prefer Woven Shirts
for dept in depts:
    print "Table Size for %s: %d" % (dept,table3[dept].shape[0])

Table Size for Woven Shirts: 107
Table Size for Knit Tops: 465
Table Size for Dresses: 111
Table Size for Pants: 254


In [83]:
#Now we check and see just how many of these N number of customers actually did "prefer" the dept the 3-month timespan said they would
for dept in depts:
    June_Users = {'Woven Shirts': [],
         'Knit Tops': [],
         'Dresses': [],
         'Pants': []}
    N=0
    for i,row in table3[dept].iterrows():
        #if N%100 == 0:
        #    print N
        #N+=1
        #print row.Ilink
        key = getHigherPref(row)
        if type(key) is list:
            for x in key:
                June_Users[x].append(row.Ilink)
        elif key == 0:
            pass
        else:
            June_Users[key].append(row.Ilink)        
    not_depts = [x for x in depts if x != dept]
    deptN = table3[dept].shape[0]
    dept_total = len(June_Users[dept])
    not_dept_total = 0
    for ndept in not_depts:
        not_dept_total += len(June_Users[ndept])
    print "Based on those that prefer %s" % dept
    print "Total Number that still 'prefer' %s: %d" % (dept,dept_total)
    print "Total Number that did not 'prefer' %s: %d" % (dept,not_dept_total)
    print "Ratio of Not_Prefer/Prefer: %f" % (not_dept_total*1.0/dept_total)
    print "Percent that kept %s as pref: %f" %(dept,dept_total*1.0/deptN)
    print ''

Based on those that prefer Woven Shirts
Total Number that still 'prefer' Woven Shirts: 80
Total Number that did not 'prefer' Woven Shirts: 30
Ratio of Not_Prefer/Prefer: 0.375000
Percent that kept Woven Shirts as pref: 0.747664

Based on those that prefer Knit Tops
Total Number that still 'prefer' Knit Tops: 393
Total Number that did not 'prefer' Knit Tops: 72
Ratio of Not_Prefer/Prefer: 0.183206
Percent that kept Knit Tops as pref: 0.845161

Based on those that prefer Dresses
Total Number that still 'prefer' Dresses: 76
Total Number that did not 'prefer' Dresses: 35
Ratio of Not_Prefer/Prefer: 0.460526
Percent that kept Dresses as pref: 0.684685

Based on those that prefer Pants
Total Number that still 'prefer' Pants: 170
Total Number that did not 'prefer' Pants: 90
Ratio of Not_Prefer/Prefer: 0.529412
Percent that kept Pants as pref: 0.669291



## 4. Look at July-Aug

In [100]:
%%time
table4 = {}
for dept in depts:
    table4[dept] = getTableRating(ilink_ex[dept],df4)
    cols = ['N_ws','n_ws','S_ws','s_ws','si_ws','D_ws','d_ws','di_ws','R_ws',
            'N_kt','n_kt','S_kt','s_kt','si_kt','D_kt','d_kt','di_kt','R_kt',
            'N_d','n_d','S_d','s_d','si_d','D_d','d_d','di_d','R_d',
            'N_p','n_p','S_p','s_p','si_p','D_p','d_p','di_p','R_p']
    table4[dept] = table4[dept].loc[~(table4[dept][cols]==0).all(axis=1)] #gets rid of customers that did not purchase anything,but have orders
    table4[dept].reset_index(drop=True,inplace=True)

CPU times: user 1min 10s, sys: 858 ms, total: 1min 10s
Wall time: 1min 16s


In [101]:
#displays first 5 rows of sorted top pref to get a sense of populations features to ratings
#Each table consists of customers that were listed as "preferred" in the categories
#Based on their "proxy rating" for May-July
#For just June month, we apply same proxy rating
pref = {'Woven Shirts': 'R_ws','Knit Tops':'R_kt','Dresses':'R_d','Pants':'R_p'}
for dept in depts:
    print "Table for %s" % dept
    display(table4[dept].sort_values(pref[dept],ascending=False).head())

Table for Woven Shirts


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
39,24256,6,6,251.94,251.94,41.99,108.00,108,18,0.90,0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0.00,0,0,0.0,0,0,0
2,5100,1,1,59.25,59.25,59.25,19.75,19.75,19.75,0.74,0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0.00,0,0,0.0,0,0,0
47,33023,1,1,35.99,35.99,35.99,24.00,24,24,0.67,0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,0.0,0,0,0,1,1,29.99,29.99,29.99,20.0,20,20,0.11
34,19763,2,2,66.48,66.48,33.24,33.50,33.5,16.75,0.67,0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0.00,0,0,0.0,0,0,0
17,9755,1,1,28.49,28.49,28.49,21.50,21.5,21.5,0.59,0,0,0.0,0,0,0.0,0,0,0,0,0,0.0,0,0,0.0,0,0,0,0,0,0.00,0,0,0.0,0,0,0


Table for Knit Tops


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
176,23327,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00,3,3.0,140.37,140.37,46.79,56.63,56.63,18.88,0.91,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
70,10723,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00,5,5.0,183.10,183.10,36.62,73.88,73.88,14.78,0.89,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
302,36070,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00,4,2.0,252.03,126.02,63.01,33.97,16.98,8.49,0.89,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
266,32805,0,0.0,0.00,0.00,0,0.00,0.00,0,0.00,5,5.0,99.70,99.70,19.94,75.25,75.25,15.05,0.85,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0
36,6699,1,0.0,20.48,10.24,20.48,19.51,9.76,19.51,0.14,3,2.0,82.38,41.19,27.46,35.61,17.80,11.87,0.81,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0.0


Table for Dresses


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
68,34202,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,2,2.0,139.98,139.98,69.99,0.00,0.00,0.00,0.89,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,24968,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,1,1.0,70.54,70.54,70.54,28.46,28.46,28.46,0.85,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
67,34039,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,49.00,12.25,49.00,0.0,0.0,0.0,0.16,4,1.0,279.99,70.00,70.00,18.00,4.50,4.50,0.74,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4063,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,2,1.0,99.73,49.86,49.86,50.25,25.12,25.13,0.72,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,20461,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,59.99,30.00,59.99,0.0,0.0,0.0,0.16,3,2.0,125.97,62.98,41.99,54.00,27.00,18.00,0.66,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Table for Pants


,Ilink,N_ws,n_ws,S_ws,s_ws,si_ws,D_ws,d_ws,di_ws,R_ws,N_kt,n_kt,S_kt,s_kt,si_kt,D_kt,d_kt,di_kt,R_kt,N_d,n_d,S_d,s_d,si_d,D_d,d_d,di_d,R_d,N_p,n_p,S_p,s_p,si_p,D_p,d_p,di_p,R_p
57,14649,0,0.0,0.0,0.0,0,0.00,0.00,0,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.00,0.00,0,0,1,1.0,63.41,63.41,63.41,25.59,25.59,25.59,0.89
25,8904,0,0.0,0.0,0.0,0,0.00,0.00,0,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.00,0.00,0,0,10,10.0,332.40,332.40,33.24,167.50,167.50,16.75,0.79
9,4434,0,0.0,0.0,0.0,0,0.00,0.00,0,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.00,0.00,0,0,4,2.0,83.98,41.99,20.99,35.99,17.99,9.00,0.78
74,19390,0,0.0,0.0,0.0,0,0.00,0.00,0,0.00,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0,0.00,0.00,0,0,6,6.0,199.44,199.44,33.24,100.50,100.50,16.75,0.72
36,11202,1,1.0,9.0,9.0,9,30.99,30.99,30.99,0.14,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1.0,13.5,13.5,13.5,46.49,46.49,46.49,0.14,3,3.0,38.24,38.24,12.75,131.73,131.73,43.91,0.71


In [104]:
#Give you the relative number of customers that prefer based on proxy rating those categories from May-July timeframe
#Example: based on purchase history from May 2017 - July 2017, the proxy rating says 108 customers prefer Woven Shirts
for dept in depts:
    print "Table Size for %s: %d" % (dept,table4[dept].shape[0])

Table Size for Woven Shirts: 55
Table Size for Knit Tops: 313
Table Size for Dresses: 74
Table Size for Pants: 145


In [53]:
#Now we check and see just how many of these N number of customers actually did "prefer" the dept the 3-month timespan said they would
for dept in depts:
    July_Users = {'Woven Shirts': [],
         'Knit Tops': [],
         'Dresses': [],
         'Pants': []}
    N=0
    for i,row in table4[dept].iterrows():
        #if N%100 == 0:
        #    print N
        #N+=1
        #print row.Ilink
        key = getHigherPref(row)
        if type(key) is list:
            for x in key:
                July_Users[x].append(row.Ilink)
        elif key == 0:
            pass
        else:
            July_Users[key].append(row.Ilink)        
    not_depts = [x for x in depts if x != dept]
    deptN = table4[dept].shape[0]
    dept_total = len(July_Users[dept])
    not_dept_total = 0
    for ndept in not_depts:
        not_dept_total += len(July_Users[ndept])
    print "Based on those that prefer %s" % dept
    print "Total Number that still 'prefer' %s: %d" % (dept,dept_total)
    print "Total Number that did not 'prefer' %s: %d" % (dept,not_dept_total)
    print "Ratio of Not_Prefer/Prefer: %f" % (not_dept_total*1.0/dept_total)
    print "Percent that kept %s as pref: %f" %(dept,dept_total*1.0/deptN)
    print ''

0
100


In [54]:
len(July_Users[dept])

91

In [58]:
deptN = len(July_Users[dept])
not_deptN = len(July_Users['Dresses']) + len(July_Users['Knit Tops']) + len(July_Users['Woven Shirts'])
print not_deptN*1.0/deptN
print deptN*1.0/table4.shape[0]

0.615384615385
0.627586206897
